In [84]:
# Import libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras
import tensorflow as tf
import warnings
import os
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
warnings.filterwarnings('ignore')

In [85]:
url = 'Datasets/2012_2019_PlatteRiverWeir_features_merged_all.csv'

dfCNN = pd.read_csv(url)

In [86]:
dfCNN

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WeirPt2X,WeirPt2Y,WwRawLineMin,WwRawLineMax,WwRawLineMean,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma
0,2012-06-09 13:15:00,2012-06-09T13:09:07,StateLineWeir_20120609_Farrell_001.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:28,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,2012-06-09 13:15:00,2012-06-09T13:10:29,StateLineWeir_20120609_Farrell_002.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:33,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,2012-06-09 13:45:00,2012-06-09T13:44:01,StateLineWeir_20120609_Farrell_003.jpg,USGS,6674500,MDT,2.96,873.0,2020-03-11T16:58:40,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,2012-06-09 14:45:00,2012-06-09T14:44:30,StateLineWeir_20120609_Farrell_004.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:47,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,2012-06-09 15:45:00,2012-06-09T15:44:59,StateLineWeir_20120609_Farrell_005.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:55,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42054,2019-10-11 09:00:00,2019-10-11T08:59:53,StateLineWeir_20191011_Farrell_409.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:53,4288,...,2446,1900,9284.0,77521.0,38385.370066,15952.029728,0.0,70085.0,37550.894823,16444.401209
42055,2019-10-11 10:00:00,2019-10-11T09:59:52,StateLineWeir_20191011_Farrell_410.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:54,4288,...,2440,1900,10092.0,74614.0,40162.989292,15467.708856,0.0,70061.0,39397.339095,16009.008049
42056,2019-10-11 11:00:00,2019-10-11T10:59:52,StateLineWeir_20191011_Farrell_411.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:55,4288,...,2447,1900,7067.0,83260.0,42095.946590,16770.357949,0.0,76335.0,41350.006568,17489.374617
42057,2019-10-11 12:00:00,2019-10-11T11:59:53,StateLineWeir_20191011_Farrell_412.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:56,4288,...,2443,1900,6283.0,83045.0,45345.490954,17498.432849,0.0,78882.0,44553.920296,18268.294896


In [87]:
input_path_train = []

for filename in os.listdir('train_test_val_images/train'):
    for path in 'train_test_val_images/train/'+dfCNN[' Filename']:
        input_path_train.append(os.path.join('train_test_val_images/train', filename, path))

In [88]:
df_train = pd.DataFrame()
df_train['images'] = input_path_train
df_train['filename'] = dfCNN[' Filename']
df_train['discharge'] = dfCNN[' Discharge']
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,images,filename,discharge
0,train_test_val_images/train\images\train_test_...,StateLineWeir_20140528_Farrell_739.JPG,2110.0
1,train_test_val_images/train\images\train_test_...,StateLineWeir_20150719_Farrell_022.JPG,1250.0
2,train_test_val_images/train\images\train_test_...,StateLineWeir_20141022_Farrell_803.JPG,359.0
3,train_test_val_images/train\images\train_test_...,StateLineWeir_20190124_Farrell_052.jpg,173.0
4,train_test_val_images/train\images\train_test_...,StateLineWeir_20130418_Farrell_718.JPG,164.0
...,...,...,...
42054,train_test_val_images/train\images\train_test_...,StateLineWeir_20190508_Farrell_165.jpg,285.0
42055,train_test_val_images/train\images\train_test_...,StateLineWeir_20190127_Farrell_145.jpg,169.0
42056,train_test_val_images/train\images\train_test_...,StateLineWeir_20180708_Farrell_098.jpg,1290.0
42057,train_test_val_images/train\images\train_test_...,StateLineWeir_20170205_Farrell_278.jpg,292.0


In [89]:
input_path_test = []

for filename in os.listdir('train_test_val_images/test'):
    for path in 'train_test_val_images/test/'+dfCNN[' Filename']:
        input_path_test.append(os.path.join('train_test_val_images/test', filename, path))

In [90]:
df_test = pd.DataFrame()
df_test['images'] = input_path_test
df_test['filename'] = dfCNN[' Filename']
df_test['discharge'] = dfCNN[' Discharge']
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,images,filename,discharge
0,train_test_val_images/test\images\train_test_v...,StateLineWeir_20141212_Farrell_675.JPG,255.0
1,train_test_val_images/test\images\train_test_v...,StateLineWeir_20160727_Farrell_859.jpg,1600.0
2,train_test_val_images/test\images\train_test_v...,StateLineWeir_20170923_Farrell_647.jpg,1210.0
3,train_test_val_images/test\images\train_test_v...,StateLineWeir_20190120_Farrell_933.jpg,178.0
4,train_test_val_images/test\images\train_test_v...,StateLineWeir_20120617_Farrell_198.jpg,1110.0
...,...,...,...
42054,train_test_val_images/test\images\train_test_v...,StateLineWeir_20160726_Farrell_837.jpg,1580.0
42055,train_test_val_images/test\images\train_test_v...,StateLineWeir_20170702_Farrell_384.jpg,1400.0
42056,train_test_val_images/test\images\train_test_v...,StateLineWeir_20140709_Farrell_514.JPG,1410.0
42057,train_test_val_images/test\images\train_test_v...,StateLineWeir_20150508_Farrell_438.JPG,249.0


In [91]:
input_path_val = []

for filename in os.listdir('train_test_val_images/val'):
    for path in 'train_test_val_images/val/'+dfCNN[' Filename']:
        input_path_val.append(os.path.join('train_test_val_images/val', filename, path))

In [92]:
df_val = pd.DataFrame()
df_val['images'] = input_path_val
df_val['filename'] = dfCNN[' Filename']
df_val['discharge'] = dfCNN[' Discharge']
df_val = df_val.sample(frac=1).reset_index(drop=True)
df_val

,images,filename,discharge
0,train_test_val_images/val\images\train_test_va...,StateLineWeir_20170710_Farrell_601.jpg,1380.0
1,train_test_val_images/val\images\train_test_va...,StateLineWeir_20190410_Farrell_327.jpg,193.0
2,train_test_val_images/val\images\train_test_va...,StateLineWeir_20180513_Farrell_423.jpg,87.1
3,train_test_val_images/val\images\train_test_va...,StateLineWeir_20130720_Farrell_767.JPG,1270.0
4,train_test_val_images/val\images\train_test_va...,StateLineWeir_20140805_Farrell_155.JPG,1540.0
...,...,...,...
42054,train_test_val_images/val\images\train_test_va...,StateLineWeir_20180712_Farrell_194.jpg,1380.0
42055,train_test_val_images/val\images\train_test_va...,StateLineWeir_20170821_Farrell_811.jpg,1280.0
42056,train_test_val_images/val\images\train_test_va...,StateLineWeir_20160731_Farrell_981.jpg,1730.0
42057,train_test_val_images/val\images\train_test_va...,StateLineWeir_20190824_Farrell_435.jpg,2870.0


In [93]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu',input_shape=(512,512,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))

In [111]:
model.compile(optimizer='adam', metrics=['mse'], loss=[tf.keras.metrics.MeanSquaredError()])

In [112]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 510, 510, 64)      1792      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 255, 255, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 253, 253, 128)     73856     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 126, 126, 128)    0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 124, 124, 256)     295168    
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 62, 62, 256)     

In [99]:
df_train.columns

Index(['images', 'filename', 'discharge'], dtype='object')

In [120]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_dataframe(
    df_train,
    # directory='train_test_val_images/',
    x_col='images',
    y_col='discharge',
    target_size=(512,512),
    batch_size=256,
    class_mode='input'
)

val_data = datagen.flow_from_dataframe(
    df_val,
    # directory='train_test_val_images/',
    x_col='images',
    y_col='discharge',
    target_size=(512,512),
    batch_size=256,
    class_mode='input'
)

Found 0 validated image filenames.
Found 0 validated image filenames.


In [110]:
history = model.fit(train_data, epochs=5, validation_data=val_data)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\alex_\anaconda3\envs\Projects\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\alex_\anaconda3\envs\Projects\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\alex_\anaconda3\envs\Projects\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\alex_\anaconda3\envs\Projects\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\alex_\anaconda3\envs\Projects\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\alex_\anaconda3\envs\Projects\lib\site-packages\keras\engine\compile_utils.py", line 184